# Data merging

### Import required packages

In [1]:
import pandas as pd
import numpy as np
import warnings

from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, MinMaxScaler

warnings.filterwarnings("ignore")

### Import required dataset into pandas dataframe

In [14]:
df1 = pd.read_csv("data/bank-full.csv", sep = ";", header = 0)
#df2 = pd.read_csv("data")
#df3 = pd.read_csv("data")

First merging

In [29]:
df1 = pd.DataFrame(df1)
merged_df = pd.merge(df1, df2, how='inner', on='id')
merged_df.shape
merged_df.columns

Index(['age', 'job', 'marital', 'education', 'default', 'balance', 'housing',
       'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'y'],
      dtype='object')

In [30]:
numerical_columns = ['age', 'balance', 'duration', 'campaign', 'pdays', 'previous']
categorical_columns = ['job', 'marital', 'education', 'default', 'housing','loan', 'contact', 'day','month', 'poutcome', 'y']

11

In [ ]:
num_pipeline = Pipeline(
                steps=[
                    ("imputer", IterativeImputer(strategy="median")),
                    ("scaler", MinMaxScaler()),
                ]
            )

cat_pipeline = Pipeline(
    #("label_mapping", FunctionTransformer(map_class_labels)),
        )

Imputers will generate synthetic data based on existing features and use it to fill up the empty cells.